In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import glob
from random import sample

In [2]:
products_df = pd.read_pickle('./prod_level_bsr_rev.pickle')

In [3]:
products_df.head()

,asin,mo_1_mean,mo_2_mean,mo_3_mean,mo_1_median,mo_2_median,mo_3_median,mo_1_min,mo_2_min,mo_3_min,...,label_after_2_yr_period_12_mo_median_bsr,label_after_1_yr_period_3_mo_min_bsr,label_after_1_yr_period_3_mo_10_perc_bsr,label_after_1_yr_period_3_mo_median_bsr,label_after_1_5_yr_period_3_mo_min_bsr,label_after_1_5_yr_period_3_mo_10_perc_bsr,label_after_1_5_yr_period_3_mo_median_bsr,label_after_2_yr_period_3_mo_min_bsr,label_after_2_yr_period_3_mo_10_perc_bsr,label_after_2_yr_period_3_mo_median_bsr
0,B00005K9DO,1.198820e+05,101592.562366,111303.227222,103538.000,93593.00,110549.50,15917.0,35518.000000,57708.000000,...,0,0,0,0,0,0,0,0,0,0
1,B0009DVYVC,6.871078e+04,14283.912847,13196.105645,56523.250,13882.25,12691.50,20931.0,7759.666667,8176.000000,...,0,1,0,0,0,0,0,0,0,0
2,B000CL8LAI,1.105162e+06,730441.677419,470000.216667,1114473.875,691677.00,468332.50,971148.0,160881.000000,172227.000000,...,0,0,0,0,0,0,0,0,0,0
3,B000H8A212,2.941590e+05,316723.588710,149149.611111,299642.250,333154.00,125755.50,117928.0,150299.000000,59737.000000,...,0,0,0,0,0,0,0,0,0,0
4,B000POZG0U,1.838216e+05,154731.102151,163585.706111,164412.000,154003.50,165687.15,90450.0,99845.000000,76813.333333,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#Features of interest: num_reviews_3_mo, revrating_mean_3_mo, reviewrating_meanverified_3_mo,
#mo_1_mean, mo_2_mean, mo_3_mean, mo_1_median, mo_2_median, mo_3_median, mo_1_min, mo_2_min, 
#mo_3_min, mean_over_3_mo, median_over_3_mo, min_over_3_mo

#Target: label_after_1_yr_period_12_mo_min_bsr,

In [5]:
for col in products_df.columns:
    print(col)

asin
mo_1_mean
mo_2_mean
mo_3_mean
mo_1_median
mo_2_median
mo_3_median
mo_1_min
mo_2_min
mo_3_min
mean_over_3_mo
median_over_3_mo
min_over_3_mo
after_1_yr_period_12_mo_min_bsr
after_1_yr_period_12_mo_10_perc_bsr
after_1_yr_period_12_mo_median_bsr
after_2_yr_period_12_mo_min_bsr
after_2_yr_period_12_mo_10_perc_bsr
after_2_yr_period_12_mo_median_bsr
after_1_yr_period_3_mo_min_bsr
after_1_yr_period_3_mo_10_perc_bsr
after_1_yr_period_3_mo_median_bsr
after_1_5_yr_period_3_mo_min_bsr
after_1_5_yr_period_3_mo_10_perc_bsr
after_1_5_yr_period_3_mo_median_bsr
after_2_yr_period_3_mo_min_bsr
after_2_yr_period_3_mo_10_perc_bsr
after_2_yr_period_3_mo_median_bsr
product_name
review_title_3_mo
review_text_3_mo
reviewvotes_num_3_mo
reviewrating_3_mo
reviewverifiedpurchase_3_mo
country_name_3_mo
review_title_6_mo
review_text_6_mo
reviewvotes_num_6_mo
reviewrating_6_mo
reviewverifiedpurchase_6_mo
country_name_6_mo
start_month
label_after_1_yr_period_12_mo_min_bsr
label_after_1_yr_period_12_mo_10_perc_bsr

In [6]:
#Number of reviews
def num_rev(x):
  return len(x['reviewrating_3_mo'])
#Creating column of number of reviews
products_df['num_reviews_3_mo']=products_df.apply(num_rev,axis=1)

In [7]:
products_df['revrating_mean_3_mo']=products_df['reviewrating_3_mo'].map(lambda x: np.mean(x))

In [8]:
#Weighted mean review ratings based on verification of review
def weight_mean_verified(x):
  if sum(x['reviewverifiedpurchase_3_mo'])==0:
    return 0
  else:
    return np.average(x['reviewrating_3_mo'],weights=x['reviewverifiedpurchase_3_mo']) 

products_df['reviewrating_meanverified_3_mo']=products_df.apply(weight_mean_verified,axis=1)

In [9]:
def num_verified_rev(x):
    y=np.array(x['reviewverifiedpurchase_3_mo'])
    return np.count_nonzero(y==True)
    #return np.count_nonzero(x['reviewverifiedpurchase_3_mo']==True)
#Creating column with number of verified reviews
products_df['num_verified_rev_3_mo']=products_df.apply(num_verified_rev,axis=1)

In [10]:
products_df['num_verified_rev_3_mo']

0        1
1        3
2        1
3        1
4        1
        ..
3686     1
3687     8
3688     3
3689    27
3690    56
Name: num_verified_rev_3_mo, Length: 3691, dtype: int64

In [11]:
products_df=products_df[["asin","num_reviews_3_mo", "revrating_mean_3_mo", "reviewrating_meanverified_3_mo", "num_verified_rev_3_mo","mo_1_mean", "mo_2_mean", "mo_3_mean", "mo_1_median", "mo_2_median", "mo_3_median", "mo_1_min", "mo_2_min", "mo_3_min", "mean_over_3_mo", "median_over_3_mo", "min_over_3_mo", "label_after_1_yr_period_12_mo_min_bsr"]]

In [12]:
products_df.head()

,asin,num_reviews_3_mo,revrating_mean_3_mo,reviewrating_meanverified_3_mo,num_verified_rev_3_mo,mo_1_mean,mo_2_mean,mo_3_mean,mo_1_median,mo_2_median,mo_3_median,mo_1_min,mo_2_min,mo_3_min,mean_over_3_mo,median_over_3_mo,min_over_3_mo,label_after_1_yr_period_12_mo_min_bsr
0,B00005K9DO,1,5.000000,5.000000,1,1.198820e+05,101592.562366,111303.227222,103538.000,93593.00,110549.50,15917.0,35518.000000,57708.000000,110925.944230,103538.00,15917.000000,0
1,B0009DVYVC,7,4.571429,4.333333,3,6.871078e+04,14283.912847,13196.105645,56523.250,13882.25,12691.50,20931.0,7759.666667,8176.000000,32063.598426,13882.25,7759.666667,1
2,B000CL8LAI,1,5.000000,5.000000,1,1.105162e+06,730441.677419,470000.216667,1114473.875,691677.00,468332.50,971148.0,160881.000000,172227.000000,768534.479576,691677.00,160881.000000,0
3,B000H8A212,1,5.000000,5.000000,1,2.941590e+05,316723.588710,149149.611111,299642.250,333154.00,125755.50,117928.0,150299.000000,59737.000000,253344.078512,299642.25,59737.000000,0
4,B000POZG0U,1,5.000000,5.000000,1,1.838216e+05,154731.102151,163585.706111,164412.000,154003.50,165687.15,90450.0,99845.000000,76813.333333,167379.453329,164412.00,76813.333333,0


In [13]:
products_df.to_pickle('./bsr_review_features.pickle')  